In [1]:
# scrape dependencies
import requests
import re
from bs4 import BeautifulSoup as bs

# data analysis dependencies
import pandas as pd
import numpy as np
import csv

# ipynb dependencies
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

# viz dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

import datetime as dt
import time

In [2]:
def parse_definition(regex_pattern, string):
    result = re.compile(regex_pattern, flags=re.MULTILINE|re.DOTALL)
    if not result.search(string):
        return "None"
    else:
        return result.search(string).group(1)

In [3]:
# set the url to scrape
url = 'https://www.ncbi.nlm.nih.gov/pubmed/?term=parkinsons+disease'
print(url)

https://www.ncbi.nlm.nih.gov/pubmed/?term=parkinsons+disease


In [4]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [5]:
# lets scrape the article titles
journals = soup.find_all("p", attrs={'class':'title'})

In [6]:
journals_len = len(journals)
print(f"There are {journals_len} journals to scrape on the first page.")

There are 20 journals to scrape on the first page.


In [7]:
for i in range(0,20):
    journals[i].text.strip()

'Now is the Critical Time for Engineered Neuroplasticity.'

"Episodic memory decline in Parkinson' s disease: relation with white matter hyperintense lesions and influence of quantification method."

"Blunted Cardiovascular Responses to Exercise in Parkinson's disease Patients: Role of the Muscle Metaboreflex."

'Age- and disease-dependent increase of the mitophagy marker phospho-ubiquitin in normal aging and Lewy body disease.'

"Inflammation and fatigue in early, untreated Parkinson's Disease."

...

"Experience of care for Parkinson's disease in European countries: A survey by the European Parkinson's Disease Association."

'Nutrition and Nutraceuticals in Neuroinflammatory and Brain Metabolic Stress: Implications for Neurodegenerative Disorders.'

"Retinal changes in Parkinson's disease and glaucoma."

'Complementary Medicine in Parkinson Disease: Once Again, Surprisingly Effective.'

"Progression Rate Associated Peripheral Blood Biomarkers of Parkinson's Disease."

In [8]:
links = soup.find_all("p",attrs={'class':'links'})
links.pop(0)

<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29948920" ref="ordinalpos=1">Similar articles</a> </p>

In [9]:
print(len(links))

19


In [10]:
for i in range (0,19):
    print(links[i])
    print("----------------------------------------------")

<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29948903" ref="ordinalpos=2">Similar articles</a> </p>
----------------------------------------------
<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29947592" ref="ordinalpos=3">Similar articles</a> </p>
----------------------------------------------
<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29947276" ref="ordinalpos=4">Similar articles</a> </p>
----------------------------------------------
<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29947088" ref="ordinalpos=5">Similar articles</a> </p>
----------------------------------------------
<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29946702" ref="ordinalpos=6">Similar articles</a> </p>
----------------------------------------------
<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_ui

In [12]:
main_url = 'https://www.ncbi.nlm.nih.gov/pubmed/'
print(main_url)

https://www.ncbi.nlm.nih.gov/pubmed/


In [37]:
# pubmed_one = str(links[0])
pubmed_one = links[0]
pubmed_one

<p class="links nohighlight"><a href="/pubmed?linkname=pubmed_pubmed&amp;from_uid=29948903" ref="ordinalpos=2">Similar articles</a> </p>